<a href="https://colab.research.google.com/github/mingun0112/Dacon/blob/main/ComputerVisionAnomalyDetection/Baseline_epoch40.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:


import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install timm


     |████████████████████████████████| 431 kB 6.5 MB/s 


In [4]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time


device = torch.device('cuda')

In [5]:
!unzip '/content/drive/MyDrive/open.zip'

Archive:  /content/drive/MyDrive/open.zip
  inflating: open/sample_submission.csv  
 extracting: open/test.zip           
  inflating: open/test_df.csv        
 extracting: open/train.zip          
  inflating: open/train_df.csv       


In [6]:
!dir

drive  open  sample_data


In [7]:
!unzip '/content/open/train.zip'

Archive:  /content/open/train.zip
  inflating: train/10000.png         
  inflating: train/10001.png         
  inflating: train/10002.png         
  inflating: train/10003.png         
  inflating: train/10004.png         
  inflating: train/10005.png         
  inflating: train/10006.png         
  inflating: train/10007.png         
  inflating: train/10008.png         
  inflating: train/10009.png         
  inflating: train/10010.png         
  inflating: train/10011.png         
  inflating: train/10012.png         
  inflating: train/10013.png         
  inflating: train/10014.png         
  inflating: train/10015.png         
  inflating: train/10016.png         
  inflating: train/10017.png         
  inflating: train/10018.png         
  inflating: train/10019.png         
  inflating: train/10020.png         
  inflating: train/10021.png         
  inflating: train/10022.png         
  inflating: train/10023.png         
  inflating: train/10024.png         
  inflating: tra

In [8]:
!unzip '/content/open/test.zip'

Archive:  /content/open/test.zip
  inflating: test/20000.png          
  inflating: test/20001.png          
  inflating: test/20002.png          
  inflating: test/20003.png          
  inflating: test/20004.png          
  inflating: test/20005.png          
  inflating: test/20006.png          
  inflating: test/20007.png          
  inflating: test/20008.png          
  inflating: test/20009.png          
  inflating: test/20010.png          
  inflating: test/20011.png          
  inflating: test/20012.png          
  inflating: test/20013.png          
  inflating: test/20014.png          
  inflating: test/20015.png          
  inflating: test/20016.png          
  inflating: test/20017.png          
  inflating: test/20018.png          
  inflating: test/20019.png          
  inflating: test/20020.png          
  inflating: test/20021.png          
  inflating: test/20022.png          
  inflating: test/20023.png          
  inflating: test/20024.png          
  inflating: test

In [9]:
from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

In [10]:
train_png = sorted(glob('train/*.png'))
test_png = sorted(glob('test/*.png'))

In [11]:
train_y = pd.read_csv("open/train_df.csv")

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
print(label_unique)
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}
print(label_unique)
train_labels = [label_unique[k] for k in train_labels]
print(train_labels)

['bottle-broken_large', 'bottle-broken_small', 'bottle-contamination', 'bottle-good', 'cable-bent_wire', 'cable-cable_swap', 'cable-combined', 'cable-cut_inner_insulation', 'cable-cut_outer_insulation', 'cable-good', 'cable-missing_cable', 'cable-missing_wire', 'cable-poke_insulation', 'capsule-crack', 'capsule-faulty_imprint', 'capsule-good', 'capsule-poke', 'capsule-scratch', 'capsule-squeeze', 'carpet-color', 'carpet-cut', 'carpet-good', 'carpet-hole', 'carpet-metal_contamination', 'carpet-thread', 'grid-bent', 'grid-broken', 'grid-glue', 'grid-good', 'grid-metal_contamination', 'grid-thread', 'hazelnut-crack', 'hazelnut-cut', 'hazelnut-good', 'hazelnut-hole', 'hazelnut-print', 'leather-color', 'leather-cut', 'leather-fold', 'leather-glue', 'leather-good', 'leather-poke', 'metal_nut-bent', 'metal_nut-color', 'metal_nut-flip', 'metal_nut-good', 'metal_nut-scratch', 'pill-color', 'pill-combined', 'pill-contamination', 'pill-crack', 'pill-faulty_imprint', 'pill-good', 'pill-pill_type',

In [12]:
batch_size = 32
img_height = 180
img_width = 180

In [13]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (256, 256))
    return img

In [14]:
train_imgs = [img_load(m) for m in tqdm(train_png)]
test_imgs = [img_load(n) for n in tqdm(test_png)]

100%|██████████| 2154/2154 [01:01<00:00, 35.16it/s]


In [15]:
pip install timm


In [16]:
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 7.4 MB/s 


In [17]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode=='train':
            augmentation = random.randint(0,2)
            if augmentation==1:
                img = img[::-1].copy()
            elif augmentation==2:
                img = img[:,::-1].copy()
        img = transforms.ToTensor()(img)
        if self.mode=='test':
            pass
        
        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = timm.create_model('efficientnet_b2', pretrained=True, num_classes=88)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [18]:
batch_size = 32
epochs = 40

# Train
train_dataset = Custom_dataset(np.array(train_imgs), np.array(train_labels), mode='train')
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

In [19]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

model = Network().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler() 



best=0
for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
        
    
    train_f1 = score_function(train_y, train_pred)

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b2_ra-bcdf34b7.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_ra-bcdf34b7.pth


epoch : 1/40    time : 50s/1945s
TRAIN    loss : 0.95494    f1 : 0.25969
epoch : 2/40    time : 49s/1876s
TRAIN    loss : 0.54049    f1 : 0.40034
epoch : 3/40    time : 49s/1819s
TRAIN    loss : 0.43090    f1 : 0.50429
epoch : 4/40    time : 50s/1787s
TRAIN    loss : 0.35035    f1 : 0.57003
epoch : 5/40    time : 50s/1749s
TRAIN    loss : 0.26426    f1 : 0.64417
epoch : 6/40    time : 50s/1711s
TRAIN    loss : 0.23120    f1 : 0.70748
epoch : 7/40    time : 50s/1660s
TRAIN    loss : 0.19178    f1 : 0.75578
epoch : 8/40    time : 50s/1608s
TRAIN    loss : 0.17251    f1 : 0.79755
epoch : 9/40    time : 50s/1558s
TRAIN    loss : 0.13471    f1 : 0.83033
epoch : 10/40    time : 50s/1506s
TRAIN    loss : 0.14196    f1 : 0.84681
epoch : 11/40    time : 50s/1456s
TRAIN    loss : 0.11153    f1 : 0.84830
epoch : 12/40    time : 50s/1405s
TRAIN    loss : 0.12091    f1 : 0.86127
epoch : 13/40    time : 50s/1354s
TRAIN    loss : 0.09759    f1 : 0.86961
epoch : 14/40    time : 50s/1303s
TRAIN    loss

In [1]:
model.eval()
f_pred = []

with torch.no_grad():
    for batch in (test_loader):
        x = torch.tensor(batch[0], dtype = torch.float32, device = device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

NameError: ignored

In [21]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

In [22]:
submission = pd.read_csv("open/sample_submission.csv")

submission["label"] = f_result

submission

,index,label
0,0,tile-glue_strip
1,1,grid-good
2,2,wood-scratch
3,3,tile-crack
4,4,tile-good
...,...,...
2149,2149,tile-gray_stroke
2150,2150,screw-good
2151,2151,grid-good
2152,2152,cable-good


In [23]:
submission.to_csv("baseline_epoch40.csv", index = False)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for labels in train_labels:
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(train_imgs[i])
    #plt.title(class_names[labels[i]])
    plt.axis("off")